In [6]:
import pandas as pd 
from difflib import SequenceMatcher as SM
df_club=pd.read_csv("Datasets\Clubs_data.csv")
df_meta=pd.read_csv("Datasets\Metadata.csv")
df_organiser=pd.read_csv("Datasets\Organisers_In_Fests.csv")
df_participants=pd.read_csv("Datasets\Participants_In_Fests.csv")

In [7]:
def is_similarity(first, second, ratio):
    return SM(None, first, second).ratio() > ratio

In [8]:
#For Club name correction
for Index, row in df_club.iterrows():
    # print(Index, len(df_club.iloc[Index]["Name"]))
    df_club.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.64):
            df_club.at[Index, "Name"] = row2["Name"]
            

In [9]:
#Removing Invalid Names in Clubs CSV
list_small_5 = []
for Index, row in df_club.iterrows():
    if len(df_club.iloc[Index]["Name"]) < 7:
        list_small_5.append(Index)
df_club = df_club.drop(df_club.index[list_small_5])

In [10]:
#For Participants Name correction
for Index, row in df_participants.iterrows():
    df_participants.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.59):
            df_participants.at[Index, "Name"] = row2["Name"]

In [11]:
#Removing Invalid Names in Participations CSV
list_small_7 = []
for Index, row in df_participants.iterrows():
    if len(df_participants.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_participants = df_participants.drop(df_participants.index[list_small_7])

In [12]:
#For Organisers Name Correction
for Index, row in df_organiser.iterrows():
    df_organiser.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.69):
            df_organiser.at[Index, "Name"] = row2["Name"]

In [13]:
#Removing Invalid Names in Organisers CSV
list_small_7 = []
for Index, row in df_organiser.iterrows():
    if len(df_organiser.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_organiser = df_organiser.drop(df_organiser.index[list_small_7])

In [14]:
#Importing Numpy and creating a Combined DF with all the data
import numpy as np
df_combined = pd.concat([df_club,df_organiser,df_participants], ignore_index=True)
df_combined['Roll No.'] = np.nan

In [15]:
#Assigning Roll No. to people for making access easier
for Index, row in df_combined.iterrows():
    for Index2, row2 in df_meta.iterrows():
        if row["Name"] == row2["Name"]:
            df_combined.at[Index, "Roll No."] = row2["ID"]
df_combined = df_combined.sort_values(by="Roll No.")
df_combined_No_Null = df_combined[df_combined["Roll No."].notnull()]

In [16]:
new_cols = ["Roll_No", "Name", "Club_Name", "Fest_Name" ,"Event", "Role"]
df_combined_No_Null=df_combined_No_Null.reindex(columns=new_cols)

In [17]:
df_combined_No_Null.head()

,Roll_No,Name,Club_Name,Fest_Name,Event,Role
1704,NaN,monet wallace,NaN,fest_2,fest_2_event_5,NaN
1395,NaN,monet wallace,NaN,fest_2,fest_2_event_3,NaN
931,NaN,monet wallace,club_2,NaN,club_2_event_1,Participant
2855,NaN,monet wallace,NaN,fest_1,fest_1_event_1,NaN
4951,NaN,monet wallace,NaN,fest_1,fest_1_event_4,NaN


In [18]:
[
    {
        'name': 'Dummy',
        'id': 'id000',
        'clubs': {
            'club_i': {
                'isOrganiser': 'Organiser',
                'club_i_event_j': {
                    'participated': False
                    },
            },
        },
        'fests': {
            'fest_i': {
                'isOrganiser': '',
                'fest_i_event_j': {
                    'participated': True
                    },
            },
        }
    }
]

[{'name': 'Dummy',
  'id': 'id000',
  'clubs': {'club_i': {'isOrganiser': 'Organiser',
    'club_i_event_j': {'participated': False}}},
  'fests': {'fest_i': {'isOrganiser': '',
    'fest_i_event_j': {'participated': True}}}}]

In [19]:
def split_clubs(df_cus_grouped):
    for (Club_Name, Event, Role), df_fests in df_cus_grouped.groupby(["Club_Name", "Event", "Role"]):
        yield {
            "Club_Name" : Club_Name,
            "Event" : Event,
            "Role" : Role,
        }

In [20]:
def pd_to_clubs_dict(df):
    for (roll_No), df_cus_grouped in df.groupby(["Roll_No"]):
        yield {roll_No : {
            "Roll Number" : roll_No,
            "Student Name" : df.loc[df["Roll_No"] == roll_No, 'Name'].iloc[0],
            "Clubs" : list(split_clubs(df_cus_grouped))
        }
    }

In [21]:
import json
profile_list = list(pd_to_clubs_dict(df_combined_No_Null))
profile_dict = {}
for sub_dict in profile_list:
    profile_dict.update(sub_dict)
print(json.dumps(profile_dict, indent=4))

{}
